In [2]:
# Run once before notebook 
import os
os.chdir("..")
os.getcwd()

'/Users/kaancaylan/Desktop/HEC/mckinsey_hackathon/methane-hackathon'

In [7]:
from utils.dataloader import LoadData
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras import layers
import datetime

In [4]:
data = LoadData(metadata_path="data/train_data/metadata.csv",
                image_data_path="data/train_data/")

X, y = data.get_train_data()
X = data.normalize_data(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42,shuffle=True)
X_train_aug, y_train_aug = data.augment_data(X_train, y_train, n_epochs=32)

/Users/kaancaylan/.pyenv/versions/3.9.6/envs/myenv/lib/python3.9/site-packages/rasterio/__init__.py:304: NotGeoreferencedWarning: Dataset has no geotransform, gcps, or rpcs. The identity matrix will be returned.
  dataset = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)


In [8]:
# Set the random seed for reproducibility
tf.random.set_seed(1234)

# Define the CNN model
model = tf.keras.Sequential([
    # Convolutional layers
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 1)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(128, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    
    # Flatten the output for fully connected layers
    layers.Flatten(),
    
    # Fully connected layers
    layers.Dense(128, activation='relu'),
    layers.Dropout(0.5),  
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(1, activation='sigmoid')  # Use 'sigmoid' for binary classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='binary_crossentropy',  # Use 'binary_crossentropy' for binary classification
              metrics=['accuracy'])

# Print the model summary to view the architecture
model.summary()

# Define early stopping
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss', 
    patience=5,           
    restore_best_weights=True)

log_dir = "logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 62, 62, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2  (None, 31, 31, 32)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 29, 29, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 14, 14, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 12, 12, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 6, 6, 128)         0

In [9]:
batch_size = 32
epochs = 100
model.fit(X_train_aug, y_train_aug, 
          epochs=epochs, 
          batch_size=batch_size, 
          validation_split=0.2,
          callbacks=[early_stopping, tensorboard_callback])

Epoch 1/100
269/269 [==============================] - 8s 28ms/step - loss: 0.4949 - accuracy: 0.7639 - val_loss: 0.3184 - val_accuracy: 0.8643
Epoch 2/100
269/269 [==============================] - 7s 28ms/step - loss: 0.2959 - accuracy: 0.8809 - val_loss: 0.1807 - val_accuracy: 0.9280
Epoch 3/100
269/269 [==============================] - 8s 28ms/step - loss: 0.1626 - accuracy: 0.9352 - val_loss: 0.0828 - val_accuracy: 0.9712
Epoch 4/100
269/269 [==============================] - 8s 28ms/step - loss: 0.0769 - accuracy: 0.9710 - val_loss: 0.0532 - val_accuracy: 0.9819
Epoch 5/100
269/269 [==============================] - 8s 28ms/step - loss: 0.0573 - accuracy: 0.9812 - val_loss: 0.0579 - val_accuracy: 0.9786
Epoch 6/100
269/269 [==============================] - 8s 28ms/step - loss: 0.0407 - accuracy: 0.9865 - val_loss: 0.0490 - val_accuracy: 0.9819
Epoch 7/100
269/269 [==============================] - 8s 29ms/step - loss: 0.0318 - accuracy: 0.9887 - val_loss: 0.0379 - val_accuracy:

In [10]:
%tensorboard --logdir logs/fit

UsageError: Line magic function `%tensorboard` not found.
